# Exercise 9 - Neural Network from Scratch

## Task
Implement a multilayer Fully Connected Network from scratch using Numpy. Use the class structure provided below. Four member functions are to be implemented:
- `forward(self, x)` for the forward prediction
- `backward(self, y)` for the backpropagation in order to compute the gradients
- `zero_grad(self)` to reset the gradients to zero (to be used before a second prediction/backpropagation)
- `step(self, lr)` to update the weights and biases with gradient descent

The most challenging task is to compute the gradients. Therefore split the implementation up into two steps:
- implement the gradient computation for a sample size of one
- implement the gradient computation for an arbitrary sample size (use the currentWeightGradients and currentBiasGradients to store the intermediate results)

To check the results, you can verify the gradients by comparing them to PyTorch. A code copying the custom neural network to PyTorch, where the automatic differentiation is provided below.

After a successful verification, use the neural network to learn a function. The training algorithm is provided below. Using the Adam optimizer implementation from exercise 2.3, the training can be improved. It is almost impossible to learn a sufficiently complex function with only gradient descent. 

Finally, compare the implementation to a PyTorch implementation.

## Learning Goals
- Gain a deeper understanding of each step in the fully connected network inference and backpropogation
- Practice matching tensor dimensions of each input and output

## Hints
Backward: 
- (3.14) $$\frac{\partial z^{(l)}_{j}}{\partial w^{(l)}_{jk}} = \frac{\partial}{\partial w^{(l)}_{jk}} \sum_h w^{(l)}_{jh} a^{(l-1)}_h + b^{(l)}_j = a^{(l-1)}_k$$ 

- (3.16) $$\frac{\partial C}{\partial b^{(l)}_j} = \frac{\partial C}{\partial z^{(l)}_{j}}\frac{\partial z^{(l)}_{j}}{\partial b^{(l)}_j} = \delta^{(l)}_j \frac{\partial z^{(l)}_{j}}{\partial b^{(l)}_j} = \delta^{(l)}_j$$

- (3.18) $$\delta^{(L)}_j = \frac{\partial C}{\partial z^{(L)}_j} = 
	\frac{\partial C}{\partial a^{(L)}_j}\frac{\partial a^{(L)}_j}{\partial z^{(L)}_j} =
	%	\frac{\partial}{\partial a^{(L)}_j} \frac{1}{2}(y_j - a^{(L)}_j)^2
	\frac{\partial C}{\partial a^{(L)}_j}\sigma'(z^{(L)}_j) = 
	-(\tilde{y}_j - \sigma(z^{(L)}_j))\sigma'(z^{(L)}_j)$$

- (3.23) 	$$\delta^{(l)}_j = \sum_k w^{(l+1)}_{kj}  \delta^{(l+1)}_k \sigma'(z^{(l)}_j)$$

Start by implementing the backward propagation for only a single sample. Subsequently, extend the implementations for an arbitrary number of samples. To this end, recall the definitions provided in
Equations 

(3.10) $$C = \frac{1}{m_{\mathcal{D}}}\sum_{i=1}^{m_{\mathcal{D}}}C_i$$

and (3.11)    $$C_i = \frac{1}{2}(\tilde{y}_i - \hat{y}_i)^2 = \frac{1}{2}(\tilde{y}_i - a_i^{(L)})^2$$

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

**neural network class**

In [ ]:
class neuralNetwork:
    def __init__(
            self, layers, activation, activationGradient, xavierInitialization=True
    ):
        self.L = len(layers)  # number of layers
        if xavierInitialization == True:
            self.weights = [
                torch.nn.init.xavier_uniform_(
                    torch.zeros((layers[i], layers[i + 1])),
                    gain=torch.nn.init.calculate_gain("sigmoid"),
                ).numpy()
                for i in range(self.L - 1)
            ]
        else:
            self.weights = [
                np.random.rand(layers[i], layers[i + 1]) for i in range(self.L - 1)
            ]
        self.biases = [np.random.rand(1, layers[i + 1]) for i in range(self.L - 1)]

        self.layerActivations = []

        self.weightGradients = [
            np.zeros((layers[i], layers[i + 1])) for i in range(self.L - 1)
        ]
        self.biasGradients = [np.zeros((1, layers[i + 1])) for i in range(self.L - 1)]

        # helper variables to store gradients per sample
        self.currentWeightGradients = [
            np.zeros((layers[i], layers[i + 1])) for i in range(self.L - 1)
        ]
        self.currentBiasGradients = [
            np.zeros((1, layers[i + 1])) for i in range(self.L - 1)
        ]

        self.activation = activation
        self.activationGradient = activationGradient

    def forward(self, x):
        raise NotImplementedError()  # your code goes here
        return a

    def backward(self, y):
        if len(self.layerActivations) > 0:  # check if forward pass has been performed
            numberOfSamples = len(self.layerActivations[0])

            # for a single sample
            if numberOfSamples == 1:
                raise NotImplementedError()  # your code goes here

            # for multiple samples using for loop over samples
            elif numberOfSamples > 1:
                for j in range(numberOfSamples):
                    raise NotImplementedError()  # your code goes here (adapt the code from for a single sample)

        else:
            print("backward propagation not possible")

    def zero_grad(self):
        self.weightGradients = [
            np.zeros((layers[i], layers[i + 1])) for i in range(self.L - 1)
        ]
        self.biasGradients = [np.zeros((1, layers[i + 1])) for i in range(self.L - 1)]

    def step(self, lr):
        for i in range(self.L - 1):
            self.weights[i] -= lr * self.weightGradients[i]
            self.biases[i] -= lr * self.biasGradients[i]

**model definition**

In [ ]:
layers = [1, 4, 4, 1]
sigmoid = lambda x: 1 / (1 + np.exp(-x))
sigmoidGradient = lambda x: sigmoid(x) * (1 - sigmoid(x))

model = neuralNetwork(layers, sigmoid, sigmoidGradient)

# input data
x = (
        np.expand_dims(np.linspace(0, 1, 1), 1) + 0.2
)  # 0.2 added, such that the first value is non-zero
y = np.sin(4 * np.pi * x) ** 2

**prediction, cost computation & gradient computation**

In [ ]:
# prediction
yPred = model.forward(x)

# cost
C = 0.5 * np.mean((yPred - y) ** 2)

# gradient
model.backward(y)

## Verification with PyTorch

**model definition and cloning of model parameters**

In [ ]:
class neuralNetworkTorch(torch.nn.Module):
    def __init__(self, layers, activationFunction=torch.nn.Sigmoid()):
        super().__init__()
        modules = []
        for i in range(len(layers) - 1):
            modules.append(torch.nn.Linear(layers[i], layers[i + 1]))
            modules.append(activationFunction)

        self.model = torch.nn.Sequential(*modules)

    def forward(self, x):
        return self.model(x)


modelTorch = neuralNetworkTorch(layers)

# copy parameters
with torch.no_grad():
    for i, param in enumerate(modelTorch.parameters()):
        if i % 2 == 0:
            param.data = torch.from_numpy(model.weights[i // 2]).to(torch.float64).t()
        else:
            param.data = torch.from_numpy(model.biases[i // 2]).to(torch.float64)

**prediction, cost computation & gradient computation**

In [ ]:
xTorch = torch.from_numpy(x).to(torch.float64)
yPredTorch = modelTorch.forward(xTorch)

CTorch = 0.5 * torch.mean((yPredTorch - torch.from_numpy(y).to(torch.float64)) ** 2)
CTorch.backward()

**gradient comparison**

In [ ]:
layer = 0
# weight gradients
print("weight:")
print(np.transpose(model.weightGradients[layer]))
print(list(modelTorch.parameters())[2 * layer].grad)

# bias gradients
print("bias:")
print(model.biasGradients[layer])
print(list(modelTorch.parameters())[2 * layer + 1].grad)

## Adam optimizer from Exercise 6 from Chapter 2

In [ ]:
class AdamOptimizer:
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None
        self.n = None
        self.t = 0

    def updateParams(self, params, grads):
        if self.m is None:
            self.m = [np.zeros_like(param) for param in params]
        if self.n is None:
            self.n = [np.zeros_like(param) for param in params]

        self.t += 1  # exponent increases with epochs
        updatedParams = []

        for p, g, m, n in zip(params, grads, self.m, self.n):
            m[:] = self.beta1 * m + (1 - self.beta1) * g
            n[:] = self.beta2 * n + (1 - self.beta2) * (g ** 2)

            mhat = m / (1 - self.beta1 ** self.t)
            nhat = n / (1 - self.beta2 ** self.t)

            updatedParams.append(p - self.lr * mhat / (np.sqrt(nhat) + self.epsilon))

        return updatedParams

## Learning with the custom neural network

**select optimizer**

In [ ]:
selectOptimizer = "gradientDescent"
# selectOptimizer = 'Adam'

**model definition**

In [ ]:
layers = [1, 20, 20, 1]
sigmoid = lambda x: 1 / (1 + np.exp(-x))
sigmoidGradient = lambda x: sigmoid(x) * (1 - sigmoid(x))

model = neuralNetwork(layers, sigmoid, sigmoidGradient)

x = np.expand_dims(np.linspace(0, 1, 40), 1)  # training grid

if selectOptimizer == "gradientDescent":
    y = x ** 2
    epochs = 10000
    lr = 1e-1
elif selectOptimizer == "Adam":
    y = np.sin(2 * np.pi * x) ** 2  # a more difficult function
    lr = 1e-2
    optimizer = AdamOptimizer(lr=lr)

**training loop**

In [ ]:
start = time.perf_counter()
for epoch in range(epochs):

    model.zero_grad()
    yPred = model.forward(x)
    C = 0.5 * np.mean((yPred - y) ** 2)
    model.backward(y)

    if selectOptimizer == "gradientDescent":
        model.step(lr)
    elif selectOptimizer == "Adam":
        updatedParameters = optimizer.updateParams(
            model.weights + model.biases, model.weightGradients + model.biasGradients
        )
        model.weights = updatedParameters[: model.L - 1]
        model.biases = updatedParameters[model.L - 1:]

    if epoch % 100 == 0:
        elapsedTime = time.perf_counter() - start
        string = "Epoch: {}/{}\t\tCost = {:.2e}\t\tElapsed time = {:2f}"
        print(string.format(epoch, epochs, C, elapsedTime))
        start = time.perf_counter()

**visualize the prediction**

In [ ]:
yPred = model.forward(x)

fig, ax = plt.subplots(figsize=(12, 6))
ax.grid()
ax.plot(x, y, "k", label="ground truth")
ax.plot(x, yPred, "r--", label="prediction")
ax.legend()
plt.show()

## Learning with PyTorch

**model definition**

In [ ]:
modelTorch = neuralNetworkTorch(layers)
xTorch = torch.from_numpy(x).to(torch.float32)
yTorch = torch.from_numpy(y).to(torch.float32)
if selectOptimizer == "gradientDescent":
    optimizer = torch.optim.SGD(modelTorch.parameters(), lr)
elif selectOptimizer == "Adam":
    optimizer = torch.optim.Adam(modelTorch.parameters(), lr)

**training loop**

In [ ]:
start = time.perf_counter()
for epoch in range(epochs):
    optimizer.zero_grad()
    yPredTorch = modelTorch.forward(xTorch)
    CTorch = 0.5 * torch.mean((yPredTorch - yTorch) ** 2)
    CTorch.backward()

    optimizer.step()

    if epoch % 100 == 0:
        elapsedTime = time.perf_counter() - start
        string = "Epoch: {}/{}\t\tCost = {:.2e}\t\tElapsed time = {:2f}"
        print(string.format(epoch, epochs, CTorch.detach(), elapsedTime))
        start = time.perf_counter()

**visualize the predictions**

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.grid()
ax.plot(x, y, "k", label="ground truth")
ax.plot(xTorch, yPredTorch.detach(), "r--", label="prediction")
ax.legend()
plt.show()